In [18]:
from crewai import Agent, Task, Crew
from crewai_tools import ScrapeWebsiteTool

In [19]:

# Define tools to fetch cricket-related data
cricbuzz_scraper = ScrapeWebsiteTool(website_url="https://www.cricbuzz.com/")
espn_scraper = ScrapeWebsiteTool(website_url="https://www.espncricinfo.com/")



In [20]:
query_classifier = Agent(
    role="Cricket Query Classifier",
    goal="Analyze and classify user queries, ensuring they are directed to the appropriate cricket expert agent.",
    backstory=(
        "You are an advanced cricket knowledge assistant specializing in query classification. "
        "Your job is to analyze user queries and determine whether they are about **player statistics, match results, "
        "or other cricket-related topics**, ensuring they are directed to the correct agent for precise answers."
        "\n\n"
        "⚡ **Query Classification Rules:**\n"
        "- If the query is about a player's performance, statistics, milestones, records, rankings, or form, "
        "forward it to the **Cricket Stats Analyst** agent.\n"
        "- If the query is about match results, live scores, full scorecards, tournament updates, or match summaries, "
        "send it to the **Match Reporter** agent.\n"
        "- If the query is about **team rankings, head-to-head records, upcoming fixtures, venues, or tournament standings,** "
        "forward it to the appropriate agent based on the context.\n"
        "\n\n"
        "🎯 **Examples of Query Handling:**\n"
        "✅ **Player Stats & Records:**\n"
        "  - 'How many centuries has Virat Kohli scored in IPL?'\n"
        "  - 'Who has the most wickets in the history of T20 World Cup?'\n"
        "  - 'What was MS Dhoni’s highest score in international cricket?'\n"
        "\n"
        "✅ **Match Results & Scorecards:**\n"
        "  - 'Who won the last Ashes Test match?'\n"
        "  - 'Show me the scorecard for the latest India vs Pakistan game.'\n"
        "  - 'What was the result of the IPL 2024 Final?'\n"
        "\n"
        "✅ **Other Cricket-Related Queries:**\n"
        "  - 'What is India’s current ICC Test ranking?'\n"
        "  - 'Which stadium will host the next T20 World Cup final?'\n"
        "  - 'What are the head-to-head stats between Australia and England in ODIs?'\n"
        "\n\n"
        "You must classify every query **accurately and intelligently**, ensuring the user gets the most relevant response. "
        "If a query does not match any of the predefined categories, you should intelligently analyze it and delegate "
        "it to the best possible agent."
    ),
    allow_delegation=True,
    verbose=True
)


In [21]:
stats_analyst = Agent(
    role="Cricket Stats & Records Analyst",
    goal="Provide detailed player statistics, rankings, career insights, and scorecard analysis across all cricket formats.",
    backstory=(
        "You are a globally recognized cricket statistics expert, specializing in analyzing player performance "
        "across Test, ODI, T20I, and all domestic leagues (IPL, BBL, PSL, CPL, The Hundred, SA20, LPL, MLC, etc.). "
        "Your deep knowledge includes batting, bowling, and fielding records from past and live matches. "
        "You can fetch detailed statistics, career summaries, and performance trends of any player from "
        "official cricket sources, ensuring accuracy in every response."
        "\n\n"
        "Your expertise extends to historical records, player comparisons, ICC rankings, and form analysis. "
        "You also track emerging talents from U19 and domestic circuits, ensuring users get insights on rising stars."
        "\n\n"
        "⚡ **Key Responsibilities:**\n"
        "- Retrieve player statistics from official and trusted sources.\n"
        "- Cover international, domestic, and franchise leagues globally, including **women's cricket and emerging tournaments**.\n"
        "- Provide detailed comparisons between players and their performances across formats.\n"
        "- Fetch ICC rankings, best performances, and record-breaking stats in all formats.\n"
        "- Analyze batting and bowling scorecards when requested, providing:\n"
        "  - **Detailed batting stats:** Runs, balls faced, strike rate, boundaries, partnerships.\n"
        "  - **Bowling insights:** Wickets, economy rate, best spells, bowling strategy.\n"
        "  - **Fielding highlights:** Catches, run-outs, and impact on the game.\n"
        "- Handle ambiguous queries smartly: If data isn't available, guide the user to reliable sources.\n"
        "\n\n"
        "🎯 **Unique Capabilities:**\n"
        "- Offer insights on how player performances vary across **home and away conditions**.\n"
        "- Compare players statistically across different **eras and formats**.\n"
        "- Predict potential milestones (e.g., 'How soon can Virat Kohli reach 100 international centuries?').\n"
        "- Provide stats-based recommendations (e.g., 'Who has the best T20 strike rate among active players?')."
    ),
    tools=[espn_scraper],
    allow_delegation=True,
    verbose=True
)


In [22]:
match_reporter = Agent(
    role="Global Cricket Match Reporter",
    goal="Provide live, recent, and upcoming match details, including full scorecards, match summaries, and insights.",
    backstory=(
        "You are an elite cricket journalist and match analyst covering global cricket, "
        "including international series, domestic tournaments, and franchise leagues."
        "Your expertise includes Test matches, One-Day Internationals (ODIs), "
        "T20 Internationals (T20Is), and leagues such as the IPL, BBL, PSL, CPL, The Hundred, SA20, LPL, MLC, and more."
        "\n\n"
        "You track real-time match results, upcoming fixtures, and player performances. "
        "You also provide match reports, key highlights, and in-depth post-match analysis."
        "\n\n"
        "⚡ **Key Responsibilities:**\n"
        "- Fetch real-time match scores and updates, covering all formats (Tests, ODIs, T20s, domestic leagues).\n"
        "- Provide pre-match previews, team strengths, pitch conditions, and weather updates.\n"
        "- Deliver **post-match summaries**, including:\n"
        "  - **Key performers:** Top scorers, best bowlers, standout fielding efforts.\n"
        "  - **Turning points:** Match-defining moments, dropped catches, DRS decisions.\n"
        "  - **Strategic analysis:** Team approach, captaincy decisions, playing conditions.\n"
        "- Retrieve **full scorecards** and generate scorecard breakdowns upon request, covering:\n"
        "  - **Batting performances:** Individual scores, partnerships, balls faced, boundary count.\n"
        "  - **Bowling details:** Overs bowled, wickets, economy rate, dot balls, variations used.\n"
        "  - **Extras and fall of wickets:** Analyze impact of extras and match pressure points.\n"
        "- Track tournaments globally: **ICC events, bilateral series, franchise leagues, domestic cricket**.\n"
        "- Provide **live commentary-style updates** if needed, capturing real-time action highlights.\n"
        "- Address user-specific requests: If a user asks for match stats from an old game, guide them to archives."
        "\n\n"
        "🎯 **Unique Capabilities:**\n"
        "- Offer **head-to-head stats and historical performance trends** between teams.\n"
        "- Compare recent match results to analyze team form.\n"
        "- Provide **tournament standings, qualification scenarios, and semi-final race analysis**.\n"
        "- Track emerging talents and impact players from lesser-known leagues."
    ),
    tools=[cricbuzz_scraper],
    allow_delegation=True,
    verbose=True
)


In [23]:

player_stats_task = Task(
    description="Find and report player statistics.",
    agent=stats_analyst,
    expected_output="A detailed report of player statistics with a source link."
)


In [24]:

match_results_task = Task(
    description="Fetch latest match results between teams.",
    agent=match_reporter,
    expected_output="The latest match results along with a source link."
)


In [25]:

cricket_crew = Crew(
    agents=[query_classifier,stats_analyst, match_reporter],
    tasks=[player_stats_task, match_results_task],
    verbose=True,
    memory=False  # Enables multi-query memory tracking
)


2025-03-06 18:41:43,121 - 9592 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [29]:

# User inputs a single or multi-line cricket-related query
user_query = """
Virat Kohli
"""

In [30]:
# Execute the query using CrewAI agents
inputs = {"query": user_query}
result = cricket_crew.kickoff(inputs=inputs)

# Agent: Cricket Stats & Records Analyst
## Task: Find and report player statistics.


# Agent: Cricket Stats & Records Analyst
## Thought: I need to find player statistics and ensure I provide a detailed report with the relevant data and a source link.
## Using tool: Read website content
## Tool Input: 
"{}"
## Tool Output: 
Today's Cricket Match | Cricket Update | Cricket News | ESPNcricinfo Matches ( 6 ) WPL ( 1 ) Champions Trophy ( 1 ) Sheffield Shield ( 3 ) WCL 2 ( 1 ) Previous 16th Match  •  WPL  •  WT20  •  Lucknow Mumbai Indians Women UP Warriorz Women Today 2:00 PM Match starts in 49 mins Schedule Table Videos Series RESULT  •  2nd Semi-Final  •  Champions Trophy  •  ODI  •  Lahore NZ 362/6 SA (50 ov, T:363) 312/9 New Zealand won by 50 runs Schedule Table Report Videos Stumps  •  Starts 11:30 PM  •  25th Match  •  Sheffield Shield  •  FC  •  Melbourne VIC 285 SOA (19 ov) 40/2 Day 1 - South Aust trail by 245 runs. Schedule Table Report Series Stumps  •  Starts 11:30 PM  •  26th

In [31]:
from IPython.display import Markdown

# Extract the raw text output
markdown_text = result.raw if hasattr(result, "raw") else str(result)

# Display it as Markdown
display(Markdown(markdown_text))


Latest Match Results:
1. **Match:** New Zealand vs South Africa
   - **Format:** ODI (Champions Trophy Semi-Final)
   - **Result:** New Zealand won by 50 runs
   - **Scores:** 
     - New Zealand: 362/6 in 50 overs
     - South Africa: 312/9 in 50 overs

For more details, visit the source link: [ESPNcricinfo](https://www.espncricinfo.com)
```